In [2]:
import SimpleGUICS2Pygame.simpleguics2pygame as simplegui
import random
import pygame.mixer

# Global constants
CANVAS_WIDTH = 600
CANVAS_HEIGHT = 400
HELICOPTER_WIDTH = 60
HELICOPTER_HEIGHT = 30
OBSTACLE_WIDTH = 40
OBSTACLE_HEIGHT = 300
HELICOPTER_SPEED = 5
OBSTACLE_SPEED = 3
MAX_OBSTACLE_SPEED = 15
BACKGROUND_MUSIC_FILE = 'helicopter.mp3' 

# Global variables
helicopter_pos = [100, CANVAS_HEIGHT // 2]
helicopter_vel = [0, 0]
obstacle_pos = [CANVAS_WIDTH, random.randrange(50, CANVAS_HEIGHT - 150)]
score = 0
game_paused = False
speed_increase_score = 2
speed_increase_counter = 0

# Initialize Pygame mixer
pygame.mixer.init()

# Load the background music
background_music = pygame.mixer.Sound(BACKGROUND_MUSIC_FILE)

# Load the background image
background_image = simplegui.load_image('bg_new.png')  # Replace 'bg_new.png' with your image file path

# Function to toggle game pause
def toggle_pause():
    global game_paused
    game_paused = not game_paused
    
    # Pause or resume the background music
    if game_paused:
        pygame.mixer.pause()
    else:
        pygame.mixer.unpause()

# Event handler for key press
def key_down(key):
    global helicopter_vel
    if key == simplegui.KEY_MAP['up']:
        helicopter_vel[1] = -HELICOPTER_SPEED
    elif key == simplegui.KEY_MAP['down']:
        helicopter_vel[1] = HELICOPTER_SPEED

# Event handler for key release
def key_up(key):
    global helicopter_vel
    if key == simplegui.KEY_MAP['up'] or key == simplegui.KEY_MAP['down']:
        helicopter_vel[1] = 0

# Event handler to draw on canvas
def draw(canvas):
    global helicopter_pos, obstacle_pos, score, game_paused, OBSTACLE_SPEED, speed_increase_counter

    # Draw the background image
    canvas.draw_image(background_image, (CANVAS_WIDTH / 2, CANVAS_HEIGHT / 2), (CANVAS_WIDTH, CANVAS_HEIGHT), (CANVAS_WIDTH / 2, CANVAS_HEIGHT / 2), (CANVAS_WIDTH, CANVAS_HEIGHT))

    if not game_paused:
        # Update helicopter position
        helicopter_pos[1] += helicopter_vel[1]
        
        # Ensure the helicopter stays within the canvas
        helicopter_pos[1] = min(max(helicopter_pos[1], 0), CANVAS_HEIGHT - HELICOPTER_HEIGHT)

        # Check for collision with top or bottom boundaries
        if helicopter_pos[1] <= 0 or helicopter_pos[1] >= CANVAS_HEIGHT - HELICOPTER_HEIGHT:
            new_game()

        # Update obstacle position and increase speed
        obstacle_pos[0] -= OBSTACLE_SPEED
        if obstacle_pos[0] < -OBSTACLE_WIDTH:
            obstacle_pos[0] = CANVAS_WIDTH
            obstacle_pos[1] = random.randrange(50, CANVAS_HEIGHT - 150)
            score += 1

            # Check for score-based speed increase
            speed_increase_counter += 1
            if speed_increase_counter >= speed_increase_score and OBSTACLE_SPEED < MAX_OBSTACLE_SPEED:
                OBSTACLE_SPEED += 1
                speed_increase_counter = 0

        # Check for collision with obstacle
        if (helicopter_pos[0] + HELICOPTER_WIDTH >= obstacle_pos[0] and
            helicopter_pos[0] <= obstacle_pos[0] + OBSTACLE_WIDTH and
            (helicopter_pos[1] <= obstacle_pos[1] or
             helicopter_pos[1] + HELICOPTER_HEIGHT >= obstacle_pos[1] + OBSTACLE_HEIGHT)):
            toggle_pause()
            new_game()

        # Draw helicopter
        canvas.draw_polygon([(helicopter_pos[0], helicopter_pos[1]),
                             (helicopter_pos[0] + HELICOPTER_WIDTH, helicopter_pos[1]),
                             (helicopter_pos[0] + HELICOPTER_WIDTH, helicopter_pos[1] + HELICOPTER_HEIGHT),
                             (helicopter_pos[0], helicopter_pos[1] + HELICOPTER_HEIGHT)],
                            1, 'White', 'White')

        # Draw obstacle
        canvas.draw_polygon([(obstacle_pos[0], 0),
                             (obstacle_pos[0] + OBSTACLE_WIDTH, 0),
                             (obstacle_pos[0] + OBSTACLE_WIDTH, obstacle_pos[1]),
                             (obstacle_pos[0], obstacle_pos[1])],
                            1, 'Red', 'Red')
        canvas.draw_polygon([(obstacle_pos[0], obstacle_pos[1] + OBSTACLE_HEIGHT),
                             (obstacle_pos[0] + OBSTACLE_WIDTH, obstacle_pos[1] + OBSTACLE_HEIGHT),
                             (obstacle_pos[0] + OBSTACLE_WIDTH, CANVAS_HEIGHT),
                             (obstacle_pos[0], CANVAS_HEIGHT)],
                            1, 'Red', 'Red')

        # Draw score
        canvas.draw_text("Score: " + str(score), (20, 30), 24, 'White')
    else:
        canvas.draw_text("Paused", (CANVAS_WIDTH // 2 - 40, CANVAS_HEIGHT // 2), 36, 'White')

# Function to start a new game
def new_game():
    global helicopter_pos, obstacle_pos, score, game_paused, OBSTACLE_SPEED, speed_increase_counter
    helicopter_pos = [100, CANVAS_HEIGHT // 2]
    obstacle_pos = [CANVAS_WIDTH, random.randrange(50, CANVAS_HEIGHT - 150)]
    score = 0
    game_paused = False
    OBSTACLE_SPEED = 3
    speed_increase_counter = 0
    background_music.play(loops=-1)  # Start playing the background music

# Function to quit the game and stop the background music
def quit_game():
    pygame.mixer.stop()
    frame.stop()

# Create frame and register event handlers
frame = simplegui.create_frame("Helicopter Game", CANVAS_WIDTH, CANVAS_HEIGHT)
frame.set_draw_handler(draw)
frame.set_keydown_handler(key_down)
frame.set_keyup_handler(key_up)
frame.add_button("New Game", new_game)
frame.add_button("Pause/Resume", toggle_pause)
frame.add_button("Quit", quit_game)  # Add a "Quit" button

# Start the game
new_game()
frame.start()
